In [4]:
import sys
import numpy as np
import cv2
from pycocotools.coco import COCO

file = "./datasets/instances_attributes_train2020.json"
dest = './datasets/yolo_data/masks/train'
db = COCO(file)
im_ids = db.getImgIds()

for im_id in im_ids:
	#print(db.imgs[im_id]['file_name'])
	ann_ids = np.array(db.getAnnIds(imgIds=im_id))
	
	cat_ids = []
	for i in ann_ids:
		cat_ids.append(db.anns[i]['category_id']+1)
	ann_ids = ann_ids[np.argsort(cat_ids)]
	
	mask = None
	for i in ann_ids:
		m = (db.anns[i]['category_id']+1)*db.annToMask(db.anns[i])
		mm = (m>0).astype(np.uint8)
		if mask is None:
			mask = m
		else:
			mask = mask*(1-mm) + m
			
	name = db.imgs[im_id]['file_name'].split('.')[0]+'.png'
	cv2.imwrite(dest+'/'+name, mask)
	print('file ' + name + ' has been written.')
	
	#cv2.imshow("mask", (mask*5.3).astype(np.uint8))
	#cv2.waitKey(0)

print("")
print("---------------------------------------------------------")
print("The list of categories are (id, name):")
for cat in db.dataset['categories']:
	print('('+str(cat['id']+1)+', '+cat['name']+')')

loading annotations into memory...
Done (t=4.24s)
creating index...
index created!
file 6ce91b2c047f7f441edd5dc17326ae17.png has been written.
file da3618863549ca1db90583f33d1d0664.png has been written.
file 23eb1448530add24409ed669467925c2.png has been written.
file 2e49eb8a326e78b76b9d92e2ce68226c.png has been written.
file 7272d5b783f8440b245e75a96990ff73.png has been written.
file 739ab7b5ce5ecd4ebd0ff2c280e93d4e.png has been written.
file d4c40a8fba2a27a6964b5a3bde178243.png has been written.
file 661bba1b15d772e7ad438b6f4b449aec.png has been written.
file c2cc67ff9334e06376ee42112ea48ea0.png has been written.
file 35f0cd519c18e2880fb243739b722c89.png has been written.
file 4ebf6ea4c376dea1b2e22e838580d39f.png has been written.
file 84aa76686256c081b7e784748a1f7dc3.png has been written.
file f6601cde98eca2f926925437005b7ec1.png has been written.
file e4cb47718ee5f9e08d686adc08c37b3a.png has been written.
file 48f7309c7d7c392533cd3f226ba871bb.png has been written.
file 4d283391fe27

In [10]:
import os
import matplotlib.pyplot as plt
import cv2


input_dir = './datasets/yolo_data/masks/val'
output_dir = './datasets/yolo_data/labels/val'

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()